In [1]:
import numpy as np

np.set_printoptions(precision=6)

In [2]:
def get_omega(L, Theta):
    omega = np.array([1/L for _ in range(L)])
    for _ in range(1000000):
        omega = omega.dot(Theta)
    return omega  

def get_normalizing_constants(L, Q, x):
    g = [[0] * L for _ in range((Q + 1))]
    for i in range(Q + 1):
        g[i][0] = x[0]**i
    g[0] = [1] * L    
    
    for i in range(1, Q + 1):
        for j in range(1, L):
            g[i][j] = g[i][j-1] + x[j] * g[i-1][j]
    return g

In [3]:
# Вариант 1

# 1. Ввод исходных данных
L = 6
Q = 15
mu = [0.4, 1, 2, 0.7, 1, 1.5]
Theta = np.array([
    [0, 0.4, 0, 0.2, 0.4, 0],
    [0, 0, 0, 0, 0.5, 0.5],
    [0.1, 0.8, 0, 0, 0, 0.1],
    [0.3, 0, 0.5, 0, 0, 0.2],
    [0, 0, 0, 0.5, 0, 0.5],
    [0, 0, 1, 0, 0, 0]
])

# Данные из примера
# L = 7
# Q = 15
# mu = [0.5, 0.6, 0.8, 1, 1.1, 1.2, 1.8]
# Theta = np.array([
#     [0, 0.5, 0.3, 0.2, 0, 0, 0],
#     [0.3, 0, 0, 0.2, 0.5, 0, 0],
#     [0, 0, 0, 0.6, 0, 0.4, 0],
#     [0.2, 0.1, 0, 0, 0.3, 0.1, 0.3],
#     [0, 0, 0, 0, 0, 0, 1],
#     [0, 0, 0.5, 0, 0, 0, 0.5],
#     [0, 0, 0, 0, 0.3, 0.7, 0]
# ])



# 2. Решение системы линейных уравнений omega*Theta = omega с условием нормировки
# omega - относительная интенсивность входящего потока требований
omega = get_omega(L, Theta)
print('omega:', omega)
print('Условие нормировки:', sum(omega))

x = [omega[i] / mu[i] for i in range(len(omega))]

# 3. Выбор масштабного коэффициента для omega[i], ограничивающего значения G(Q,L)
C = sum(x) / sum([i**2 for i in x])
print('Масштабный коэффициент:', C)

# 4. Вычисление нормализующей константы G(Q,L) и значения величин g(Y,L), Y = 1,..,Q по формуле (1.3) с (1.4)
g = get_normalizing_constants(L, Q, x)

# 5. Вычисление характеристик систем сети по формулам (1.2)

# вероятность того, что в системах m и более требований
PM = [[0] * L for _ in range((Q + 1))]
# вероятность того, что в системах ровно m требований
Pm = [[0] * L for _ in range((Q + 1))]
# м.о. числа требований в системах
s = [0] * L
# м.о. числа занятых приборов в системах
h = [0] * L
# интенсивности входного потока требований в системах
lambdas = [0] * L
# м.о. длительности пребывания требований в системах
u = [0] * L
# коэффициенты использования систем
psy = [0] * L

for i in range(Q + 1):
    for j in range(L):
        PM[i][j] = x[j]**i * (g[Q-i][L-1] / g[Q][L-1])
        Pm[i][j] = (x[j]**i / g[Q][L-1]) * (g[Q-i][L-1] - x[j] * g[Q-i-1][L-1])

for i in range(L):
    for j in range(1, Q + 1):
        s[i] += x[i]**j * ((g[Q-j][L-1] / g[Q][L-1]))
    h[i] = x[i] * ((g[Q-1][L-1] / g[Q][L-1]))
    lambdas[i] = h[i] * mu[i]
    u[i] = s[i] / lambdas[i]
    psy[i] = lambdas[i] / mu[i]
    
# 6. Вывод результатов:

print('\nСтационарные вероятности пребывания в системах m или более требований')
print(' '*40, 'Номер системы')
print('Кол.треб.', ' '*5, '1', ' '*7, '2', ' '*7, '3', ' '*7, '4', ' '*7,'5', ' '*7, '6')
for i in range(Q + 1):
    print(i, end='\t| ')
    for j in range(L):
        print('{:07}'.format(round(PM[i][j], 5)), end=' | ')
    print('\n', '-'*68)
        
print('\nСтационарные вероятности пребывания в системах ровно m требований')
print(' '*40, 'Номер системы')
print('Кол.треб.', ' '*5, '1', ' '*7, '2', ' '*7, '3', ' '*7, '4', ' '*7,'5', ' '*7, '6')
for i in range(Q + 1):
    print(i, end='\t| ')
    for j in range(L):
        print('{:07}'.format(round(Pm[i][j], 5)), end=' | ')
    print('\n', '-'*68)
    
print('\nСтационарные характеристики сети')
print('Система', ' '*8, 's', ' '*16, 'u', ' '*12, 'lambdas', ' '*10, 'psy', sep='')
for i in range(L):
    print(i, end='\t|')
    print(round(s[i], 5), '\t|', round(u[i], 5), '\t|', round(lambdas[i], 5), '\t|', round(psy[i], 5), '\t|')
    print('-'*73)

omega: [0.050553 0.235071 0.268562 0.078989 0.137757 0.229068]
Условие нормировки: 1.0000000000000027
Масштабный коэффициент: 6.230671745288869

Стационарные вероятности пребывания в системах m или более требований
                                         Номер системы
Кол.треб.       1         2         3         4         5         6
0	| 00001.0 | 00001.0 | 00001.0 | 00001.0 | 00001.0 | 00001.0 | 
 --------------------------------------------------------------------
1	| 0.53039 | 0.98653 | 0.56354 | 0.47356 | 0.57813 | 0.64089 | 
 --------------------------------------------------------------------
2	| 0.27983 | 0.96811 | 0.31591 | 0.22308 | 0.33247 | 0.40858 | 
 --------------------------------------------------------------------
3	| 0.14659 | 0.94327 | 0.17583 | 0.10434 | 0.18983 | 0.25862 | 
 --------------------------------------------------------------------
4	| 0.07605 | 0.91027 | 0.09692 | 0.04833 | 0.10736 | 0.16213 | 
 --------------------------------------------------------